<a href="https://colab.research.google.com/github/rezzie-rich/colab-notebooks/blob/main/Mistral_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Training Script for Mistral 7B with PEFT LoRA

In [ ]:
# Setup and Installation
!pip install -q transformers datasets bitsandbytes-cuda11 peft huggingface_hub
from huggingface_hub import notebook_login

# Authenticate with Hugging Face
notebook_login()

# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, merge_adapter
import time
import torch

# Base Model Configuration
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Advanced: Sequential Training Option (Uncomment for sequential training)
# datasets_list = ["dataset1_name", "dataset2_name", "dataset3_name"]
# sequential_training = True

# For Non-Sequential Training
sequential_training = False
dataset_name = "your_dataset_name_here"

# Configure LoRA with Default Options
lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"])
model = prepare_model_for_kbit_training(model, lora_config)

# Advanced: Custom LoRA Configuration (Uncomment if needed)
# lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none")

# Hyperparameter Configuration
training_args = {
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 16,
    "num_train_epochs": 3,
}

# Training Loop
def run_training(dataset_name):
    dataset = load_dataset(dataset_name)
    training_args_modified = TrainingArguments(
        output_dir=f"./model_output_{dataset_name}",
        **training_args,
        fp16=True,
        gradient_checkpointing=True
    )
    trainer = Trainer(
        model=model,
        args=training_args_modified,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"]
    )
    trainer.train()
    model.save_pretrained(f"checkpoint_after_{dataset_name}")
    print(f"Training on {dataset_name} completed.")

# Execute Training
if sequential_training:
    for ds in datasets_list:
        run_training(ds)
        print(f"Taking a break before next dataset...")
        time.sleep(300)  # 5-minute break
else:
    run_training(dataset_name)

# Upload the Model to Hugging Face
trainer.push_to_hub("Your_HF_Repo_Name")

# Advanced: Merging LoRA weights into the base model (Uncomment if needed)
# merge_adapter(model, "adapter_name")

# Additional Notes
# Ensure data preprocessing and training configurations are appropriate for your use case.
# Regularly evaluate model performance using validation datasets.
# Consider data augmentation and regularization techniques to prevent overfitting.